# Objective

Perform the exploratory data analysis (EDA) to find insights in the AWS pricing data

# Code

## Load libs

In [ ]:
import sys
sys.path.append('..')

import random
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from src.data.helpers import load_aws_dataset

## Input params

In [ ]:
interim_dir = '../data/interim'
in_fname = 'step_1_aws_filtered_sample.csv.zip'
compression = 'zip'

In [ ]:
# Papermill parameters injection ... do not delete!

## Load data

In [ ]:
file = f'{interim_dir}/{in_fname}'
data = load_aws_dataset(file)
print(data.shape)
data.head()

## Data wrangling

Let's find something interesting in the data!
- Look for most volatile instances, i.e., with more price changes, thus to avoid them;
- Least volatile instances;
- Longer price update times;

### Data check-up for nulls and missing values

Assumptions:

- Considered region: us-east-1a (Virginia);
- Check presence of null columns (i.e., there is no price change on that);

In [ ]:
%%time

df = data.query('AvailabilityZone == "us-east-1a"')\
         .drop('AvailabilityZone', axis=1)

print(df.shape)

# Pivot table to change a wide format for the data. Thus, we can remove
# instances that do not have any price update.
# Dropping MultiIndex column 'SpotPrice' as there is no use for it.
pvt = df.pivot_table(index=['Timestamp'], 
                     columns=['InstanceType'])\
        .droplevel(0, axis=1)

pvt.head()

In [ ]:
# Checking if there is any column with only 'NaN'
# Returns None, meaning that all 
pvt.isna().all(axis=0).loc[lambda x: x.isna()]

In [ ]:
# Cross-check to see if this is correct. Getting a sample of confirm this
# using instance 'a1.2xlarge'
pvt['a1.2xlarge'].dropna().head()

In [ ]:
# Picking random instance and checking if the values are not null
# just for sanity check.
for i in range(5):
    rand_instance = random.randint(0, len(pvt.columns))
    tmp = pvt.iloc[rand_instance].dropna().head()
    print(tmp)

### Most volatile instances

In [ ]:
# Now getting the most volatile instances
most_volatiles = pvt.count().sort_values(ascending=False).nlargest(10)
most_volatiles

In [ ]:
# Let's quickly plot to see the pricing trends
# Some normalization is required:
# 1. Remove rows with only NaN (not columns, otherwise it will remove all pricing!);
# 2. There are gaps in the pricing. This happens because if there is no pricing
# update, then there is not price capture. Thus, we can safely use backwards fill
# to fill the missing values

fig, ax = plt.subplots(figsize=(12, 6))

pvt.loc[:, most_volatiles.index.to_list()]\
    .dropna(how='all', axis=0)\
    .fillna(method='bfill').plot(ax=ax)

ax.set_title('Top 10 most volatile instances')
ax.set_ylabel('Hourly Price (USD)')
ax.legend(loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.35))

### Least volatile instances

In [ ]:
# Now getting the least volatile instances
least_volatiles = pvt.count().sort_values(ascending=False).nsmallest(10)
least_volatiles

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

pvt.loc[:, least_volatiles.index.to_list()]\
    .dropna(how='all', axis=0)\
    .fillna(method='bfill').plot(ax=ax)

ax.set_title('Top 10 least volatile instances')
ax.set_ylabel('Hourly Price (USD)')
ax.legend(loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.35))